# Week 8 Exercise: The Price is Right - Autonomous Deal-Hunting AI

## Overview
This notebook implements a complete autonomous agentic AI system that:
- Scans online deals from RSS feeds
- Estimates fair market prices using multiple AI/ML models
- Identifies great deals by comparing listed vs estimated prices
- Sends push notifications for good opportunities
- Displays everything in a Gradio UI

## Architecture
- **SpecialistAgent**: Fine-tuned LLM deployed on Modal
- **FrontierAgent**: RAG + GPT-4o-mini/DeepSeek
- **RandomForestAgent**: ML model on embeddings
- **EnsembleAgent**: Weighted combination of all pricers
- **ScannerAgent**: RSS feed deal scraper
- **MessagingAgent**: Push notifications
- **PlanningAgent**: Orchestrates everything

---


In [ ]:
# Core imports
import os
import sys
import json
import pickle
import logging
from dotenv import load_dotenv
from pathlib import Path

# Add parent directory to path to access week8 modules
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

print(f"Working directory: {Path.cwd()}")
print(f"Parent directory added to path: {parent_dir}")


In [ ]:
# Environment setup
load_dotenv(override=True)

# Verify required environment variables
required_vars = ['OPENAI_API_KEY', 'HF_TOKEN']
optional_vars = ['DEEPSEEK_API_KEY', 'PUSHOVER_USER', 'PUSHOVER_TOKEN']

print("Required environment variables:")
for var in required_vars:
    status = "SET" if os.getenv(var) else "MISSING"
    print(f"  {var}: {status}")

print("\nOptional environment variables:")
for var in optional_vars:
    status = "SET" if os.getenv(var) else "NOT SET"
    print(f"  {var}: {status}")


## Modal Setup

Before proceeding, ensure Modal is configured:

1. If this is your first time, uncomment and run the next cell to set up Modal
2. This will open a browser for authentication
3. Alternatively, run `modal setup` from command line in an activated environment


In [ ]:
# Modal Authentication - Run this to authenticate
# This will open a browser for you to sign in to Modal and create a token

!modal token new


In [ ]:
# Check if Modal is configured and import it
import modal
from pathlib import Path

print(f"Modal version: {modal.__version__}")

# Check if Modal token exists
modal_config = Path.home() / ".modal.toml"
if modal_config.exists():
    print("Modal configuration found - you're all set!")
else:
    print("WARNING: Modal configuration not found. You need to run 'modal setup' first.")
    print("Please follow the instructions in the cell above.")


## Configure HuggingFace Secret in Modal

Before deploying, you need to set up your HuggingFace token as a secret in Modal:

1. Go to https://modal.com and sign in
2. Navigate to **Secrets** in the sidebar
3. Click **Create new secret**
4. Select **Hugging Face**
5. Name it **hf-secret** (important: this is referenced in the code)
6. Add your HF_TOKEN value
7. Save the secret


## Review the Pricer Service Configuration

Let's examine the Modal deployment configuration:


In [ ]:
# Read and display the pricer_service.py configuration
pricer_service_path = parent_dir / "pricer_service.py"

with open(pricer_service_path, 'r') as f:
    content = f.read()
    
# Show the key configuration details
print("Pricer Service Configuration:")
print("="*50)
for line in content.split('\n')[:30]:
    if any(keyword in line for keyword in ['BASE_MODEL', 'HF_USER', 'RUN_NAME', 'GPU', 'FINETUNED_MODEL', 'REVISION']):
        print(line)


In [ ]:

# modal deploy ../pricer_service2.py

!modal deploy ../pricer_service2.py


In [ ]:
# Test the deployed pricer service
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

test_description = "Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio"

print(f"Testing pricer with: {test_description}")
print("\nCalling Modal (this may take 30 seconds on first call as container wakes up)...")

result = pricer.price.remote(test_description)

print(f"\nEstimated price: ${result:.2f}")


In [ ]:
# Import the SpecialistAgent
from agents.specialist_agent import SpecialistAgent

# Initialize logging to see agent messages
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("Initializing SpecialistAgent...")
specialist = SpecialistAgent()
print("\nAgent ready!")


In [ ]:
# Test the SpecialistAgent with multiple products
test_products = [
    "iPad Pro 2nd generation with 256GB storage",
    "Sony WH-1000XM5 wireless noise-cancelling headphones",
    "Nintendo Switch OLED model with neon controllers"
]

print("Testing SpecialistAgent with sample products:\n")
for product in test_products:
    price = specialist.price(product)
    print(f"Product: {product}")
    print(f"Estimated Price: ${price:.2f}")
    print("-" * 70)


In [ ]:
# Additional imports for RAG
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import chromadb
from huggingface_hub import login

# Import items and testing modules
from items import Item
from testing import Tester

print("RAG imports complete")


In [ ]:
# Set up constants
DB = "products_vectorstore"

# Log in to HuggingFace
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

print(f"Vector database name: {DB}")
print("HuggingFace login successful")


## Load Training Data

We need the `train.pkl` and `test.pkl` files from Week 6. These files contain the curated product data.

**Options:**
1. Copy them from your `week6` folder to `week8/philip` folder
2. Or download from: https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW

Place the files in the `week8/philip` directory before running the next cell.


In [ ]:
# Try to load from current directory first, then from week6
with open('train.pkl', 'rb') as file:
        train = pickle.load(file)
with open('test.pkl', 'rb') as file:
        test = pickle.load(file)
print(f"Loaded from current directory")

print(f"\nTraining set: {len(train):,} items")
print(f"Test set: {len(test):,} items")
print(f"\nSample item: {train[0].title}")
print(f"Price: ${train[0].price:.2f}")


## Initialize ChromaDB

ChromaDB will store our product embeddings for fast similarity search.


In [ ]:
# Initialize ChromaDB client
client = chromadb.PersistentClient(path=DB)

# Check if collection exists and delete it if needed (for fresh start)
collection_name = "products"
existing_collections = client.list_collections()

if collection_name in [col.name for col in existing_collections]:
    print(f"Deleting existing collection: {collection_name}")
    client.delete_collection(collection_name)

# Create new collection
collection = client.create_collection(collection_name)
print(f"Created collection: {collection_name}")


## Initialize SentenceTransformer

We'll use `all-MiniLM-L6-v2` which maps text to 384-dimensional vectors. It's fast and runs locally.


In [ ]:
# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Test it with a sample text
test_vector = model.encode(["iPad Pro with 256GB storage"])[0]
print(f"Model loaded successfully")
print(f"Vector dimensions: {len(test_vector)}")
print(f"Sample vector (first 10 values): {test_vector[:10]}")


## Helper Function: Extract Product Description

We need to extract clean product descriptions from our Item objects.


In [ ]:
# Helper function to extract description from Item
def description(item):
    """Extract the product description without the question and price"""
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

# Test it
sample_desc = description(train[0])
print(f"Sample description ({len(sample_desc)} chars):")
print(sample_desc[:200] + "..." if len(sample_desc) > 200 else sample_desc)


## Populate the Vector Database

Now we'll vectorize and store all products in ChromaDB.

**Options:**
- Full dataset: 400,000 products (takes ~30-45 minutes)
- Subset: 20,000 products (takes ~3-5 minutes, still gives great results)

Uncomment your preferred option in the next cell.


In [ ]:
# NUMBER_OF_DOCUMENTS = len(train)  # Full dataset (~400k)
NUMBER_OF_DOCUMENTS = 20000  # Smaller subset (faster, still effective)

print(f"Will process {NUMBER_OF_DOCUMENTS:,} documents")
print(f"Processing in batches of 1000...")
print(f"Estimated time: {NUMBER_OF_DOCUMENTS // 1000 * 7} seconds")
print("\nStarting vectorization...")


In [ ]:
# Populate ChromaDB with product vectors
for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    batch_items = train[i: i+1000]
    
    documents = [description(item) for item in batch_items]
    
    vectors = model.encode(documents).astype(float).tolist()
    
    metadatas = [{"category": item.category, "price": item.price} for item in batch_items]
    
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

print(f"\nComplete! Added {NUMBER_OF_DOCUMENTS:,} products to the vector database.")


In [ ]:
# Test query: Find wireless headphones
test_query = "Sony wireless noise-cancelling headphones"

query_vector = model.encode([test_query])

results = collection.query(
    query_embeddings=query_vector.astype(float).tolist(),
    n_results=5
)

print(f"Query: '{test_query}'")
print(f"\nTop 5 similar products:\n")

for i, (doc, metadata) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
    print(f"{i}. Price: ${metadata['price']:.2f} | Category: {metadata['category']}")
    print(f"   Description: {doc[:100]}...")
    print()


In [ ]:
# Test with another query
test_query2 = "gaming laptop with RTX graphics card"

query_vector2 = model.encode([test_query2])
results2 = collection.query(
    query_embeddings=query_vector2.astype(float).tolist(),
    n_results=5
)

print(f"Query: '{test_query2}'")
print(f"\nTop 5 similar products:\n")

for i, (doc, metadata) in enumerate(zip(results2['documents'][0], results2['metadatas'][0]), 1):
    print(f"{i}. Price: ${metadata['price']:.2f} | Category: {metadata['category']}")
    print(f"   Description: {doc[:100]}...")
    print()


In [ ]:
# Import the FrontierAgent
from agents.frontier_agent import FrontierAgent

print("FrontierAgent imported successfully")


In [ ]:
# Initialize the FrontierAgent with our ChromaDB collection
frontier = FrontierAgent(collection)

print("FrontierAgent initialized and ready!")


## Test FrontierAgent with Sample Products

Let's test the FrontierAgent with a few products and see how it performs!


In [ ]:
# Test FrontierAgent with sample products
test_products_frontier = [
    "Apple iPad Pro 12.9-inch with 256GB storage and Apple Pencil support",
    "Sony WH-1000XM5 wireless noise-cancelling headphones with 30-hour battery",
    "Nintendo Switch OLED model with vibrant 7-inch screen and neon Joy-Con controllers"
]

print("Testing FrontierAgent with sample products:\n")
print("="*80)

for product in test_products_frontier:
    print(f"\nProduct: {product}")
    print("-"*80)
    
    # Get price estimate
    estimate = frontier.price(product)
    
    print(f"FrontierAgent Estimate: ${estimate:.2f}")
    print("="*80)


In [ ]:
# Compare both agents on the same products
comparison_products = [
    "Wireless gaming mouse with RGB lighting",
    "USB-C charging cable 6 feet braided",
    "Bluetooth speaker waterproof portable"
]

print("Agent Comparison: SpecialistAgent vs FrontierAgent\n")
print("="*80)

for product in comparison_products:
    print(f"\nProduct: {product}")
    print("-"*80)
    
    # Get predictions from both agents
    specialist_price = specialist.price(product)
    frontier_price = frontier.price(product)
    
    print(f"Specialist (Fine-tuned LLM): ${specialist_price:.2f}")
    print(f"Frontier (RAG + GPT-4o-mini): ${frontier_price:.2f}")
    print(f"Difference: ${abs(specialist_price - frontier_price):.2f}")
    print("="*80)


In [ ]:
# Evaluate on a small sample of test data
num_test_samples = 10  # Small sample to keep it fast

print(f"Evaluating both agents on {num_test_samples} test samples...\n")

specialist_errors = []
frontier_errors = []

for i in range(num_test_samples):
    item = test[i]
    actual_price = item.price
    desc = description(item)
    
    # Get predictions
    specialist_pred = specialist.price(desc)
    frontier_pred = frontier.price(desc)
    
    # Calculate errors
    specialist_error = abs(specialist_pred - actual_price)
    frontier_error = abs(frontier_pred - actual_price)
    
    specialist_errors.append(specialist_error)
    frontier_errors.append(frontier_error)
    
    print(f"Item {i+1}: {item.title[:50]}...")
    print(f"  Actual: ${actual_price:.2f}")
    print(f"  Specialist: ${specialist_pred:.2f} (error: ${specialist_error:.2f})")
    print(f"  Frontier: ${frontier_pred:.2f} (error: ${frontier_error:.2f})")
    print()

# Calculate average errors
avg_specialist_error = np.mean(specialist_errors)
avg_frontier_error = np.mean(frontier_errors)

print("="*80)
print("RESULTS:")
print(f"Specialist Agent - Average Error: ${avg_specialist_error:.2f}")
print(f"Frontier Agent - Average Error: ${avg_frontier_error:.2f}")
print(f"Better performer: {'Specialist' if avg_specialist_error < avg_frontier_error else 'Frontier'}")
print("="*80)


## Understand How FrontierAgent Works

Let's peek inside to see how the FrontierAgent uses RAG to build context.


In [ ]:
# Let's see what similar products the FrontierAgent finds
test_product = "MacBook Pro 14-inch with M2 chip and 16GB RAM"

print(f"Test product: {test_product}\n")
print("="*80)

# Find similar products (this is what FrontierAgent does internally)
documents, prices = frontier.find_similars(test_product)

print("\nSimilar products found by RAG:\n")
for i, (doc, price) in enumerate(zip(documents, prices), 1):
    print(f"{i}. ${price:.2f}")
    print(f"   {doc[:150]}...")
    print()

# Now get the actual price prediction
final_price = frontier.price(test_product)
print("="*80)
print(f"\nFinal FrontierAgent prediction: ${final_price:.2f}")
print("="*80)


# Phase 4: Train Random Forest Model

In this phase, we'll:
1. Extract embeddings from ChromaDB 
2. Train a Random Forest Regressor on the embeddings
3. Evaluate its performance
4. Save the model as `random_forest_model.pkl`
5. Test the RandomForestAgent

Random Forest works directly on the vector embeddings, learning patterns without needing an LLM!


In [ ]:
# Additional imports for ML
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

print("ML imports complete")


In [ ]:
# Extract all data from ChromaDB
result = collection.get(include=['embeddings', 'documents', 'metadatas'])

# Convert to numpy arrays
vectors = np.array(result['embeddings'])
documents = result['documents']
prices = np.array([metadata['price'] for metadata in result['metadatas']])

print(f"Extracted data from ChromaDB:")
print(f"  Vectors shape: {vectors.shape}")
print(f"  Number of products: {len(documents):,}")
print(f"  Price range: ${prices.min():.2f} - ${prices.max():.2f}")
print(f"  Mean price: ${prices.mean():.2f}")


## Train Random Forest Model

Now we'll train a Random Forest Regressor on the embeddings to predict prices.


In [ ]:
# Train Random Forest
print("Training Random Forest model...")
print("This may take 1-2 minutes...\n")

rf_model = RandomForestRegressor(
    n_estimators=100,      # Number of trees
    max_depth=20,          # Max depth of each tree
    random_state=42,       # For reproducibility
    n_jobs=-1,             # Use all CPU cores
    verbose=1              # Show progress
)

# Train the model
rf_model.fit(vectors, prices)

print("\nRandom Forest training complete!")


In [ ]:
# Evaluate on training data
train_predictions = rf_model.predict(vectors)

# Calculate metrics
train_mse = mean_squared_error(prices, train_predictions)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(prices, train_predictions)

print("Random Forest - Training Set Performance:")
print("="*80)
print(f"Root Mean Squared Error (RMSE): ${train_rmse:.2f}")
print(f"R² Score: {train_r2:.4f}")
print(f"Mean Absolute Error: ${np.mean(np.abs(prices - train_predictions)):.2f}")
print("="*80)


## Test on Sample Products

Let's test the Random Forest directly with some product descriptions.


In [ ]:
# Test Random Forest with sample products
rf_test_products = [
    "Wireless Bluetooth headphones with noise cancellation",
    "USB-C laptop charger 65W power adapter",
    "Mechanical gaming keyboard RGB backlit"
]

print("Testing Random Forest model:\n")
print("="*80)

for product_desc in rf_test_products:
    # Encode the description
    product_vector = model.encode([product_desc])
    
    # Predict price
    predicted_price = max(0, rf_model.predict(product_vector)[0])
    
    print(f"Product: {product_desc}")
    print(f"Random Forest Prediction: ${predicted_price:.2f}")
    print("-"*80)


## Save the Random Forest Model

Save the trained model so the RandomForestAgent can use it.


In [ ]:
# Save the model
model_path = 'random_forest_model.pkl'
joblib.dump(rf_model, model_path)

print(f"Random Forest model saved to: {model_path}")
print(f"File size: {os.path.getsize(model_path) / 1024 / 1024:.2f} MB")


## Test the RandomForestAgent

Now let's use the RandomForestAgent class which loads and uses our saved model.


In [ ]:
# Import and initialize RandomForestAgent
from agents.random_forest_agent import RandomForestAgent

rf_agent = RandomForestAgent()
print("RandomForestAgent initialized!")


In [ ]:
# Test RandomForestAgent
agent_test_products = [
    "Apple AirPods Pro with active noise cancellation",
    "Samsung Galaxy S23 smartphone 128GB",
    "LG 55-inch 4K OLED smart TV"
]

print("Testing RandomForestAgent:\n")
print("="*80)

for product in agent_test_products:
    price = rf_agent.price(product)
    print(f"Product: {product}")
    print(f"RandomForestAgent Prediction: ${price:.2f}")
    print("-"*80)


In [ ]:
# Compare all three agents
comparison_products_all = [
    "Dell XPS 15 laptop with Intel i7 and 16GB RAM",
    "Sony PlayStation 5 console with controller",
    "Bose QuietComfort 45 wireless headphones"
]

print("THREE-WAY AGENT COMPARISON\n")
print("="*80)

for product in comparison_products_all:
    print(f"\nProduct: {product}")
    print("-"*80)
    
    specialist_price = specialist.price(product)
    frontier_price = frontier.price(product)
    rf_price = rf_agent.price(product)
    
    print(f"Specialist (Fine-tuned LLM):  ${specialist_price:>8.2f}")
    print(f"Frontier (RAG + GPT-4o):      ${frontier_price:>8.2f}")
    print(f"RandomForest (ML on vectors): ${rf_price:>8.2f}")
    print(f"Average:                      ${np.mean([specialist_price, frontier_price, rf_price]):>8.2f}")
    print("="*80)


In [ ]:
# Additional imports for ensemble
from sklearn.linear_model import LinearRegression
import pandas as pd

print("Ensemble imports ready")
print(f"Pandas version: {pd.__version__}")


## Collect Predictions from All Three Agents

We'll get predictions from all three agents on a subset of test data to train the ensemble.

**Note**: This will take several minutes as we need to call the Modal API and OpenAI API for each test sample.


In [ ]:
# Collect predictions from all three agents
# Using a subset to keep training time reasonable
NUM_ENSEMBLE_SAMPLES = 50  # Adjust if you want more/fewer samples

print(f"Collecting predictions from all three agents on {NUM_ENSEMBLE_SAMPLES} test samples...")
print("This will take a few minutes...\n")

specialist_predictions = []
frontier_predictions = []
rf_predictions = []
actual_prices = []

for i in tqdm(range(NUM_ENSEMBLE_SAMPLES)):
    item = test[i]
    desc = description(item)
    
    # Get predictions from each agent
    spec_pred = specialist.price(desc)
    front_pred = frontier.price(desc)
    rf_pred = rf_agent.price(desc)
    
    specialist_predictions.append(spec_pred)
    frontier_predictions.append(front_pred)
    rf_predictions.append(rf_pred)
    actual_prices.append(item.price)

print(f"\nCollected {NUM_ENSEMBLE_SAMPLES} predictions from each agent!")


## Prepare Training Data for Ensemble

Create a DataFrame with all the predictions and engineered features.


In [ ]:
# Create training data for ensemble
ensemble_data = pd.DataFrame({
    'Specialist': specialist_predictions,
    'Frontier': frontier_predictions,
    'RandomForest': rf_predictions,
})

# Add min and max features (helps the ensemble understand uncertainty)
ensemble_data['Min'] = ensemble_data[['Specialist', 'Frontier', 'RandomForest']].min(axis=1)
ensemble_data['Max'] = ensemble_data[['Specialist', 'Frontier', 'RandomForest']].max(axis=1)

print("Ensemble training data:")
print(ensemble_data.head(10))
print(f"\nShape: {ensemble_data.shape}")


## Train the Ensemble Model

Train a Linear Regression to learn the best way to combine the three agent predictions.


In [ ]:
# Train ensemble model
print("Training Ensemble Model...\n")

ensemble_model = LinearRegression()
ensemble_model.fit(ensemble_data, actual_prices)

# Make predictions
ensemble_predictions = ensemble_model.predict(ensemble_data)

# Evaluate
ensemble_mae = np.mean(np.abs(np.array(actual_prices) - ensemble_predictions))
ensemble_rmse = np.sqrt(mean_squared_error(actual_prices, ensemble_predictions))
ensemble_r2 = r2_score(actual_prices, ensemble_predictions)

print("Ensemble Model Performance:")
print("="*80)
print(f"Mean Absolute Error: ${ensemble_mae:.2f}")
print(f"Root Mean Squared Error: ${ensemble_rmse:.2f}")
print(f"R² Score: {ensemble_r2:.4f}")
print("="*80)


## Analyze Ensemble Weights

Let's see how the ensemble weights each agent's predictions.


In [ ]:
# Show the learned weights
print("Ensemble Model Weights:")
print("="*80)
for feature, coef in zip(ensemble_data.columns, ensemble_model.coef_):
    print(f"{feature:15s}: {coef:8.4f}")
print(f"{'Intercept':15s}: {ensemble_model.intercept_:8.4f}")
print("="*80)
print("\nInterpretation: Higher weights mean that agent has more influence on the final prediction.")


## Compare Individual vs Ensemble Performance

Let's see if the ensemble actually improves over individual agents!


In [ ]:
# Compare performance of all models
spec_mae = np.mean(np.abs(np.array(actual_prices) - np.array(specialist_predictions)))
front_mae = np.mean(np.abs(np.array(actual_prices) - np.array(frontier_predictions)))
rf_mae = np.mean(np.abs(np.array(actual_prices) - np.array(rf_predictions)))

print("Performance Comparison (Mean Absolute Error):")
print("="*80)
print(f"Specialist Agent:     ${spec_mae:8.2f}")
print(f"Frontier Agent:       ${front_mae:8.2f}")
print(f"RandomForest Agent:   ${rf_mae:8.2f}")
print(f"Ensemble Agent:       ${ensemble_mae:8.2f}  <-- Combined!")
print("="*80)

best_individual = min(spec_mae, front_mae, rf_mae)
improvement = ((best_individual - ensemble_mae) / best_individual) * 100

if ensemble_mae < best_individual:
    print(f"\nEnsemble improves over best individual by {improvement:.1f}%")
else:
    print(f"\nEnsemble is within {abs(improvement):.1f}% of best individual")


## Save the Ensemble Model

Save the trained ensemble so the EnsembleAgent can use it.


In [ ]:
# Save the ensemble model
ensemble_model_path = 'ensemble_model.pkl'
joblib.dump(ensemble_model, ensemble_model_path)

print(f"Ensemble model saved to: {ensemble_model_path}")
print(f"File size: {os.path.getsize(ensemble_model_path) / 1024:.2f} KB")


## Test the EnsembleAgent

Now let's use the EnsembleAgent class which orchestrates all three agents!


In [ ]:
# Import and initialize EnsembleAgent
from agents.ensemble_agent import EnsembleAgent

print("Initializing EnsembleAgent (this creates all three sub-agents)...")
ensemble_agent = EnsembleAgent(collection)
print("EnsembleAgent ready!")


In [ ]:
# Test the EnsembleAgent
ensemble_test_products = [
    "Apple MacBook Air M2 chip 13-inch with 256GB SSD",
    "Nintendo Switch OLED with Mario Kart bundle",
    "Dyson V15 cordless vacuum cleaner"
]

print("Testing EnsembleAgent (calls all 3 agents internally):\n")
print("="*80)

for product in ensemble_test_products:
    print(f"\nProduct: {product}")
    print("-"*80)
    
    # The ensemble agent calls all three agents and combines their predictions
    final_price = ensemble_agent.price(product)
    
    print(f"Final Ensemble Prediction: ${final_price:.2f}")
    print("="*80)


In [ ]:
# Import deal-related classes
from agents.deals import ScrapedDeal, DealSelection, Deal, Opportunity
from agents.scanner_agent import ScannerAgent

print("Scanner imports complete")


## Test RSS Feed Scraping

First, let's see what deals are available from RSS feeds.


In [ ]:
# Fetch deals from RSS feeds
print("Fetching deals from RSS feeds...")
print("This scrapes from multiple deal websites (DealNews, etc.)")
print("May take 1-2 minutes...\n")

deals = ScrapedDeal.fetch(show_progress=True)

print(f"\nFetched {len(deals)} deals from RSS feeds!")


In [ ]:
# Look at a sample deal
sample_deal = deals[2] if deals else None

if sample_deal:
    print(f"\nSample Deal #{1}:")
    print("="*80)
    print(sample_deal.describe())
    print("="*80)


## Test the ScannerAgent

The ScannerAgent uses GPT-4o-mini to intelligently select and parse the best deals.


In [ ]:
# Initialize ScannerAgent
scanner = ScannerAgent()

print("ScannerAgent initialized!")


In [ ]:
# Use the scanner to select and parse the best deals
print("ScannerAgent is analyzing deals with GPT-4o-mini...")
print("This will select the 5 best deals with clear prices and descriptions\n")

# Empty memory means it won't filter out any deals
selected_deals = scanner.scan(memory=[])

if selected_deals:
    print(f"ScannerAgent selected {len(selected_deals.deals)} high-quality deals:\n")
    print("="*80)
    
    for i, deal in enumerate(selected_deals.deals, 1):
        print(f"\nDeal {i}:")
        print(f"  Description: {deal.product_description[:100]}...")
        print(f"  Price: ${deal.price:.2f}")
        print(f"  URL: {deal.url}")
        print("-"*80)
else:
    print("No deals found or parsed successfully")


In [ ]:
# Import messaging and planning agents
from agents.messaging_agent import MessagingAgent
from agents.planning_agent import PlanningAgent

print("Messaging and Planning imports complete")


## Setup Pushover 

For push notifications, you'll need Pushover:
1. Sign up at https://pushover.net (free tier available)
2. Create an application to get your PUSHOVER_TOKEN
3. Get your PUSHOVER_USER key from your account
4. Add both to your `.env` file

If you don't have Pushover set up, the agent will still work - it just won't send notifications.


In [ ]:
# Initialize MessagingAgent
messenger = MessagingAgent()

print("MessagingAgent initialized!")


In [ ]:
# Test push notification (only if you have Pushover configured)

pushover_configured = os.getenv('PUSHOVER_USER') and os.getenv('PUSHOVER_TOKEN')

if pushover_configured:
    print("Testing push notification...")
    messenger.push("Test from Price is Right system!")
    print("Check your phone/device for the notification!")
else:
    print("Pushover not configured - skipping notification test")
    print("To enable: add PUSHOVER_USER and PUSHOVER_TOKEN to your .env file")


## Initialize the PlanningAgent

The PlanningAgent coordinates all the other agents!


In [ ]:
# Initialize PlanningAgent
planner = PlanningAgent(collection)

print("PlanningAgent initialized and ready to coordinate all agents!")


## Test Single Deal Processing

Let's test how the planner processes a single deal.


In [ ]:
# Create a test deal
test_deal = Deal(
    product_description="Sony WH-1000XM5 wireless noise-cancelling headphones, premium sound quality, 30-hour battery life",
    price=299.99,
    url="https://example.com/deal"
)

print("Testing PlanningAgent with a single deal...")
print(f"\nDeal: {test_deal.product_description[:80]}...")
print(f"Listed Price: ${test_deal.price:.2f}\n")

# Process the deal
opportunity = planner.run(test_deal)

print(f"\nResults:")
print("="*80)
print(f"Deal Price:     ${opportunity.deal.price:.2f}")
print(f"Estimate:       ${opportunity.estimate:.2f}")
print(f"Discount:       ${opportunity.discount:.2f}")
print(f"Good deal?:     {'YES!' if opportunity.discount > 50 else 'Not quite'}")
print("="*80)


## Run Complete Planning Cycle

Now let's run a full planning cycle: scan, price, and identify opportunities!


In [ ]:
# Run a full planning cycle
print("Running full planning cycle...")
print("This will:")
print("  1. Scan RSS feeds for deals")
print("  2. Parse with GPT-4o-mini")
print("  3. Price each deal with EnsembleAgent")
print("  4. Identify best opportunity")
print("  5. Send notification if discount > $50\n")

print("="*80)

best_opportunity = planner.plan(memory=[])

if best_opportunity:
    print(f"\nBEST OPPORTUNITY FOUND:")
    print("="*80)
    print(f"Product: {best_opportunity.deal.product_description[:100]}...")
    print(f"Deal Price: ${best_opportunity.deal.price:.2f}")
    print(f"Estimated Value: ${best_opportunity.estimate:.2f}")
    print(f"Potential Savings: ${best_opportunity.discount:.2f}")
    print(f"URL: {best_opportunity.deal.url}")
    print("="*80)
    
    if pushover_configured:
        print("\nNotification sent to your device!")
else:
    print("\nNo opportunities found with discount > $50")


In [ ]:
# Import Gradio and framework
import gradio as gr
from deal_agent_framework import DealAgentFramework

print("Gradio and framework imports complete")


## Initialize the Agent Framework

The DealAgentFramework manages the entire system including memory persistence.


In [ ]:
# Initialize the agent framework
agent_framework = DealAgentFramework()
agent_framework.init_agents_as_needed()

print("Agent framework initialized and ready!")


In [ ]:

with gr.Blocks(title="The Price is Right", fill_width=True) as ui:
    
    gr.Markdown('<div style="text-align: center;font-size:24px">The Price is Right - Deal Hunting AI</div>')
    gr.Markdown('<div style="text-align: center;font-size:14px">Autonomous agent framework for finding great deals</div>')
    
    # Display current opportunities
    opportunities_display = gr.Dataframe(
        headers=["Description", "Price", "Estimate", "Discount", "URL"],
        label="Opportunities Found",
        wrap=True,
        column_widths=[4, 1, 1, 1, 2],
        row_count=10,
    )
    
    # Button to trigger a scan
    scan_button = gr.Button("Run Scan Cycle", variant="primary")
    status_text = gr.Textbox(label="Status", lines=3)
    
    def run_scan():
        try:
            # Run the planning cycle (returns full memory list)
            memory_before_count = len(agent_framework.memory)
            all_opportunities = agent_framework.run()
            
            if all_opportunities and len(all_opportunities) > 0:
                # Create table data from all opportunities
                table_data = [[
                    opp.deal.product_description[:80] + "...",
                    f"${opp.deal.price:.2f}",
                    f"${opp.estimate:.2f}",
                    f"${opp.discount:.2f}",
                    opp.deal.url
                ] for opp in all_opportunities]
                
                # Check if new opportunity was added
                if len(all_opportunities) > memory_before_count:
                    latest = all_opportunities[-1]
                    status = f"New opportunity found! Discount: ${latest.discount:.2f}"
                else:
                    status = "Scan complete. No new opportunities found (discount < $50)"
                    
                return table_data, status
            else:
                status = "Scan complete. No opportunities found."
                return gr.update(), status
        except Exception as e:
            import traceback
            error_details = traceback.format_exc()
            return gr.update(), f"Error: {str(e)}\n\nDetails:\n{error_details}"
    
    scan_button.click(run_scan, outputs=[opportunities_display, status_text])



In [ ]:
# Launch the UI
ui.launch(inbrowser=True)
